# Data Preparation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ast

path = "../data_raw/charging_sessions.csv"
df = pd.read_csv(path)



Get an overview over the data through info & head

In [ ]:
df.info()
df.head()

## Clean Data

### Drop the index column "Unnamed: 0"

The column has 66450 integer values. However, because of the missing column name it is not clear what this integer value is supposed to represent other than maybe an identifier. We already have the column "id" as our identifier, therefore the column "Unnamed: 0" does not provide any value to us and we drop it.

In [ ]:
# 1) Drop explicit index column
df = df.drop(columns=['Unnamed: 0'])

## Format Values

### Parse all datetimes in the Localtime America/Los_Angeles for consistency & drop timezone


In [ ]:
unique_timezones = df["timezone"].dropna().unique()
print("Unique timezones:", unique_timezones)

Finding: All records share a single timezone (America/Los_Angeles).

As all values of this feature are the same and do not provide any value for us by knowing the city, we drop this column.

In [ ]:
df = df.drop(columns=['timezone'])

The three timestamp columns are therefore parsed as UTC and converted to this local timezone for consistency.

In [ ]:
time_cols = ["connectionTime", "disconnectTime", "doneChargingTime"]
local_timezone = unique_timezones[0]
for col in time_cols:
    df[col] = pd.to_datetime(df[col], utc=True, errors="coerce")
    df[col] = df[col].dt.tz_convert(local_timezone)

### Format time data by adding year, month etc.

In [ ]:
#Create new columns for year, month, day, hour, day of the week and the season
df['year'] = df['connectionTime'].dt.year
df['month'] = df['connectionTime'].dt.month
df['day'] = df['connectionTime'].dt.day
df['hour'] = df['connectionTime'].dt.hour
df['dayofweek'] = df['connectionTime'].dt.dayofweek

def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'
    
df['season'] = df['month'].apply(get_season)

### Parse all values of kwhDelivered in floats

In [ ]:
# dont think this is necessary as dytpe is already float
print(df['kWhDelivered'].dtype)

df['kWhDelivered'] = pd.to_numeric(df['kWhDelivered'], errors='coerce')

### Use string dytpe and format string for categorical columns

In [ ]:
cat_cols = ["siteID", "spaceID", "stationID", "id", "sessionID"]
for col in cat_cols:
    # print(f'Unique values in {col}: ', df[col].nunique(), f' vs. Unique values in {col} after cleaning: ', df[col].astype(str).str.strip().str.lower().nunique())
    df[col] = df[col].astype(str).str.strip().str.lower()

## Handle Missing Values

In [ ]:
print(df.isnull().sum())

Finding: The columns doneChargingTime, userID, and userInputs have missing values.

### Handle missing doneChargingTime values

In [ ]:
mask_na = df["doneChargingTime"].isna() & df["kWhDelivered"].gt(0)
df_na = df.loc[mask_na].copy()
print('COUNT missing doneChargingTime with kWhDelivered > 0: ', mask_na.sum())

There are 4,088 records where doneChargingTime is missing. Every one of these records has positive kWhDelivered. This indicates that the charging process occurred normally, but the system failed to log doneChargingTime. These records will be analyzed further to verify that imputing doneChargingTime = disconnectTime is reasonable.

In [ ]:
df_na["duration_h"] = (
    df_na["disconnectTime"] - df_na["connectionTime"]
).dt.total_seconds() / 3600
df_na[["duration_h", "kWhDelivered"]].describe(include="all")

The missing doneChargingTime sessions show plausible charging behavior:

* Average session duration ≈ 4.7 hours, median ≈ 3.9 hours

* Average energy delivered ≈ 14 kWh, with max ≈ 78 kWh

* No negative or zero durations appear, indicating correct chronological order

Although setting doneChargingTime = disconnectTime  represents the latest possible end of charging and may overestimate the true charging duration for some sessions, it is the only defensible imputation given the available data.

Dropping these sessions would remove 6.15% of all sessions and introduce systematic bias, while estimating an earlier timestamp would require unavailable information (EV model, SOC, charging curve).

Therefore, imputing doneChargingTime = disconnectTime is the most sound approach.

In [ ]:
# If doneChargingTime is missing but energy was delivered -> set to disconnectTime
df.loc[mask_na, "doneChargingTime"] = df.loc[mask_na, "disconnectTime"]

### Handle missing userID values



In [ ]:
print(df.isnull().sum()['userID'])
print('FRACTION missing userID: ', df['userID'].isna().sum() / len(df))

As one forth of the userID values are missing it is not plausible to delete these rows. Rather it is better to use placeholder value indicating that no value was given. This placeholder value will be -1.

In [ ]:
df.fillna({'userID': '-1'}, inplace=True)

## Check for invalid values

### Check for invalid doneChargingTime entries

To ensure temporal consistency, `doneChargingTime` must be between `connectionTime` and `disconnectTime`.  

In [ ]:
mask_1 = df["doneChargingTime"] < df["connectionTime"]
mask_2 = df["doneChargingTime"] > df["disconnectTime"]
mask_3 = (df["doneChargingTime"] == df["connectionTime"]) & (df["kWhDelivered"] > 0)
print('COUNT doneChargingTime < connectionTime: ', mask_1.sum())
print('COUNT doneChargingTime > disconnectTime: ', mask_2.sum())
print('COUNT doneChargingTime == connectionTime with energyDelivered > 0: ', mask_3.sum())

df_invalid = df[mask_1 | mask_2].copy()


We identified three types of temporal inconsistencies in the dataset:

* `doneChargingTime < connectionTime`: 27 cases  
* `doneChargingTime > disconnectTime`: 4,692 cases  
* `doneChargingTime == connectionTime with energyDeliverivered > 0`: 2 cases 

We then quantified how far the invalid `doneChargingTime` values deviate from their valid bounds:

In [ ]:
# How many invalid doneChargingTime values are "close" (e.g. within 300 seconds)?

threshold = 300  # seconds

late_off  = (df.loc[mask_2, "doneChargingTime"] - df.loc[mask_2, "disconnectTime"]).dt.total_seconds().abs()


print(f"LATE   cases: {len(late_off)} sessions")
print(f"  -> { (late_off <= threshold).mean():.2%} within {threshold} seconds of disconnectTime")


These results show that the most inconsistencies are extremely small (typically just a few seconds or minutes), indicating minor logging delays rather than invalid sessions. Therefore the most transparent and sound correction is to **clip `doneChargingTime` into the valid interval**. This restores temporal consistency while preserving all meaningful charging sessions for subsequent analysis.

In [ ]:
# Clip doneChargingTime to [connectionTime, disconnectTime]
df.loc[mask_2, "doneChargingTime"] = df.loc[mask_2, "disconnectTime"]

For the case `doneChargingTime < connectTime` it is not resonable to clip `doneChargingTime` into the interval & set it at the connectTime. This is due top the fact that the charging duration would then be 0 with positive kWhDelivered, which doesn't make any sense. Because there are just 27 cases we can drop these instances. For the case `doneChargingTime == connectionTime with energyDeliverivered > 0` it is also reasonable dueto the same reasoning to drop these 2 instances

In [ ]:
mask_del = mask_1 & mask_3
df.drop(df[mask_del].index, inplace=True)

### Check for sessions where the disconnectTime is before connection

In [ ]:
# Sessions where disconnectTime < connectionTime
print('COUNT disconnectTime < connectionTime: ', (df["disconnectTime"] < df["connectionTime"]).sum())

There are no sessions where disconnectTime < connectionTime

### Check for sessions where kWhDelivered is negative

In [ ]:
# Sessions where kWhDelivered is negative
print('COUNT negative energy rows: ', (df["kWhDelivered"] < 0).sum())


There are no sessions where kWhDelivered is negative

### Check for sessions where duration is negative

In [ ]:
# Check for session with negative duration
duration_h = (df["disconnectTime"] - df["connectionTime"]).dt.total_seconds() / 3600.0
print('COUNT of sessions <= 0 h: ', (duration_h <= 0).sum())


There are no session where duration is negative

## Handle duplicated rows

### Handle duplicates on sessionID

In [ ]:
dup_mask = df["sessionID"].duplicated(keep=False)
num_duplicated_sessions = dup_mask.sum()
print("Rows with duplicated sessionID:", num_duplicated_sessions)

Each `sessionID` should represent exactly one charging session, but **2,826 rows** appear more than once.    
These duplicates indicate repeated logging of the same session and must be consolidated.

We first remove duplicate `sessionID`s (keeping the latest record per session) and then run checks on `id` and the physical key (`stationID`, `connectionTime`) to
confirm that no duplicates remain.

In [ ]:
sort_cols = ["sessionID", "connectionTime"]
df = df.sort_values(sort_cols)
df = df.drop_duplicates(subset=["sessionID"], keep="last")

In [ ]:
# Check duplicates on id
print(df["id"].duplicated().sum())

# Check duplicates on physical key (spaceID, connectionTime)
print(df.duplicated(subset=["spaceID", "connectionTime"], keep=False).sum())

df[df.duplicated(subset=["spaceID", "connectionTime"], keep=False)]


## Adding needed Features

### Adding Session & Charging duration

In [ ]:
# Compute session and charging durations to choose a reporting unit
session_duration_minutes = (df["disconnectTime"] - df["connectionTime"]).dt.total_seconds() / 60
charging_duration_minutes = (df["doneChargingTime"] - df["connectionTime"]).dt.total_seconds() / 60

# Summary stats in minutes and hours
stats = pd.DataFrame({
    "mean_minutes": [session_duration_minutes.mean(), charging_duration_minutes.mean()],
    "median_minutes": [session_duration_minutes.median(), charging_duration_minutes.median()],
    "mean_hours": [session_duration_minutes.mean() / 60, charging_duration_minutes.mean() / 60],
    "median_hours": [session_duration_minutes.median() / 60, charging_duration_minutes.median() / 60],
}, index=["session_duration", "charging_duration"]).round(2)
print("Dauer-Statistiken (Minuten/Stunden):")
print(stats)

We median minutes of both duration is higher than 60. They have a median hour of around 6 & 3 hours witch make it plausible to use hours for the duration.

In [ ]:
df['session_duration'] = (df["disconnectTime"] - df["connectionTime"]).dt.total_seconds() / 3600
df['charging_duration'] = (df["doneChargingTime"] - df["connectionTime"]).dt.total_seconds() / 3600

### Adding registeredUser

In [ ]:
df['isRegisteredUser'] = df['userID'] != '-1'

## Handle User Inputs

The last column of the given dataset contains user inputs. We also have to check for missing/errornous values here. Best approach would be to load the user input data into a seperate dataframe containing the sessionID as foreign key.

(STILL HAVE TO BE JOINED TOGETHER LATER ON)

In [ ]:
# Parse user inputs (read the string in the column "user inputs" into usable python objects) 
df_parsed = df.copy()
df_parsed["userInputs"] = df_parsed["userInputs"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else None)

# 1. Create sessions dataframe
sessions_df = df_parsed.drop(columns=["userInputs"]).copy()

# Create user input dataframe and explode user input column into dictonaries
tmp = df_parsed[["sessionID", "userInputs"]].copy()
tmp = tmp.explode("userInputs")
tmp = tmp.dropna(subset=["userInputs"])

# Normalize user input dictionaries into columns
user_input_df = pd.json_normalize(tmp["userInputs"])
user_input_df["sessionID"] = tmp["sessionID"].values

user_input_df.info()
user_input_df.head()

Finding: The resulting dataframe has a total of 9 columns and 61663 rows. There are no missing values.

### Convert timestamps to same format as charging_sessions entries

We know that all records share a single timezone (America/Los_Angeles).

The two timestamp columns are therefore parsed as UTC and converted to this local timezone for consistency, just as before with the charging sessions.

In [ ]:
time_cols = ["modifiedAt", "requestedDeparture"]
local_timezone = unique_timezones[0]
for col in time_cols:
    user_input_df[col] = pd.to_datetime(user_input_df[col], utc=True, errors="coerce")
    user_input_df[col] = user_input_df[col].dt.tz_convert(local_timezone)

## Save processed dataframes

First, we have to merge them back together.

In [ ]:
# TODO

# Think about a better way of merging, because the resulting dataset does not contain a suitable id column as primary key
# We now have a row for each user input, duplicating the sessionIDs and IDs for multiple user inputs in the same session

merged_df = pd.merge(sessions_df, user_input_df, on='sessionID', how='left')
merged_df.head()
merged_df.info()

Finding: We now have multiple userID columns
We can check if the IDs align for each entry (setting the value for missing userIDs to "" to ensure that only existing values are being compared):

In [ ]:
print((merged_df["userID_x"].fillna("") == merged_df["userID_y"].fillna("")).all())

This returns true, therefore one of the two user_id columns can be dropped. We drop userID_y and rename userID_x.

In [ ]:
merged_df = merged_df.drop(columns = ['userID_y'])
merged_df.rename(columns={"userID_x": "userID"}, inplace=True)

Now we can save that dataframe into a new csv for later use.

In [ ]:
output_path = "../data_processed/"
merged_df.to_csv(output_path + "charging_sessions_processed.csv", index=False)

In [ ]:
import pickle

# Save prepared dataframes as pickle files for use in other notebooks
output_path = "../data_processed/"

with open(output_path + "df_prepared.pkl", "wb") as f:
    pickle.dump(df, f)

with open(output_path + "merged_df_prepared.pkl", "wb") as f:
    pickle.dump(merged_df, f)

print("DataFrames saved as pickle files:")
print(f"  - {output_path}df_prepared.pkl")
print(f"  - {output_path}merged_df_prepared.pkl")